# PILes
Simple wrapper for Python PIL to draw multiple shapes on the same picture.

In [1]:
# External imports. Matplotlib for simple displays
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from PIL import Image

In [2]:
# Imports from this module.
from piles import PILe, ImageDraws
import Chatoyant
from distributions import Spiral, Parametric, Circular, RandomCoords, Uniform
from functions import Distance, Angle, Modify

## The module is separated in 4 main modules: 

#### distributions.py to generate coordinates of complex distributions. 
Classes must output coordinates in the [-1, 1] range.

#### functions.py to generate weights for the coordinates
Classes must ouput weights in the [0, 1] range.

#### Chatoyant.py to generate complex colormaps
To be used by PILes. 

#### piles.py for the interface with PIL
To draw complex images

## 1. Distributions
Distributions output a tuple of (x, y) coordinates in the [-1, 1] range.

In [ ]:
# Available distributions for now.
distribs = [Spiral().golden(), Spiral().archimedean(), Spiral().quadratic(), Spiral().patterned(),
                Parametric().sunflower(), Parametric().lissajous(), Parametric().gear(),
                Circular().uniform(), Circular().polygon(), Circular().star(),
                RandomCoords().disk(), RandomCoords().normal(), RandomCoords().circular(), RandomCoords().rectangular(),
                Uniform().square(), Uniform().hexagon()]
    
names = ['Golden Spiral', 'Archimedean Spiral', 'Quadratic Spiral', 'Patterned Spiral',
         'Sunflower', 'Lissajous', 'Gears',
         'Circular Uniform', 'Polygon', 'Star',
         'Random Disk', 'Random Normal', 'Random Circular', 'Random Rectangular',
         'Uniform Square', 'Uniform Hexagonal']

fig, axes = plt.subplots(figsize=(12, 12), nrows=4, ncols=4, sharex=True, sharey=True)
ax = axes.ravel()

# Let's plot them
for i, (dist, name) in enumerate(zip(distribs, names)):

    ax[i].scatter(*dist)
    ax[i].set_aspect('equal')
    ax[i].set_title(name)
    ax[i].axis('off')

fig.suptitle('Available distributions', fontsize=18)
fig.tight_layout(pad=1.2)

Each distribution has parameters to change the final shape. Example here when changing the parameter k of quadratic spirals.

In [ ]:
fig, axes = plt.subplots(figsize=(16,4), ncols=4, nrows=1)
ax = axes.ravel()
[(x.axis('off'), 
  x.set_aspect('equal'),) for x in ax]

for i, k in enumerate([5, 10, 15, 20]):
    x, y = Spiral(n=100).quadratic(k=k)
    ax[i].scatter(x, y)
    ax[i].set_title(f'k={k}')

fig.suptitle('Changing Parameter k of Quadratic Spirals', fontsize=18)
fig.tight_layout(pad=1.2)

And for sunflower distributions:

In [ ]:
fig, axes = plt.subplots(figsize=(16,4), ncols=4, nrows=1)
ax = axes.ravel()
[(x.axis('off'), 
  x.set_aspect('equal'),) for x in ax]

for i, a in enumerate([1, 2, 3, 4]):
    x, y = Parametric(n=100).sunflower(alpha=a)
    ax[i].scatter(x, y)
    ax[i].set_title(f'alpha={a}')

fig.suptitle('Changing parameter alpha of Sunflower distributions', fontsize=18)
fig.tight_layout(pad=1.2)

## 2. Functions
Functions to create weights from the coordinates generated by the distributions. It has to outputs values between 0 and 1.
Available: dependent to distance from the centre of the distribution and dependent to the angle theta of the point.

In [ ]:
fig, axes = plt.subplots(figsize=(16,4), ncols=4, nrows=1)
ax = axes.ravel()
[(x.axis('off'), 
  x.set_aspect('equal'),) for x in ax]

x, y = Parametric(n=500).sunflower() # 500 points with sunflower distribution

# Examples of use
sizes = Distance(x, y).wave(waves=2) # 2 waves depending on distance from centre
sizes2 = Angle(x, y).wave(waves=3) # 3 waves depending on the theta angle of the point

ax[0].scatter(x, y, s=20, c = 'red')
ax[0].set_title('Sunflower Distribution')

ax[1].scatter(x, y, s=sizes*20, c = 'red')
ax[1].set_title('Sizes depending on distance')

ax[2].scatter(x, y, s=sizes2*20, c = 'red')
ax[2].set_title('Sizes depending on angle')

ax[3].scatter(x, y, s=sizes*sizes2*20, c = 'red') # Simply multiply the sizes to combine them 
ax[3].set_title('Sizes depending on distance & angle');

fig.suptitle('Size can be modified by distance from centre and theta angle', fontsize=18)
fig.tight_layout(pad=1.2)


## 3. Chatoyant
Module made to easily create color maps.

They can be created from Matplotlib colormaps:

In [ ]:
Chatoyant.ColorMap().from_matplotlib('inferno', n=20)

Inverted, added together.

In [ ]:
cmap = Chatoyant.ColorMap().from_matplotlib('RdBu', n=20)
cmap + cmap.invert()

They can also be created from list of known color names or (R, G, B) tuples and then be interpolated:

In [ ]:
Chatoyant.ColorMap().from_list(color_list=[(255, 0, 0), 'green', 'blue', (0, 0, 0)]).extend(20)

Everything can be combined to produce complex colormaps.

In [ ]:
cmap1 = Chatoyant.ColorMap().from_matplotlib('inferno', n=20)
cmap2 = Chatoyant.ColorMap().from_matplotlib('viridis', n=50).invert()

cmap3 = (cmap1 + cmap2).loop(n=2).set_name('Shiny one')
cmap3

See a Notebook for more details how to use it: https://github.com/Sylvain-Deposit/Chatoyant/blob/main/Chatoyant%20Example.ipynb

## 4.PILes
Wrapper made on top of PIL to draw multiple shapes with different sizes, colors, alphas, etc... in a single call.
It is fully compatible with PIL, uses the same format and outputs the same kind of images.
However, it has to be fed with a PILe dataclass containing the coordinates, sizes, colors, angles, etc... generated by the distributions, functions, colormaps modules.

As it is using PIL, remember that color values are in (R, G, B) with values between 0 and 255, alphas are between 0 and 255, angles between 0 and 360 degrees, contour widths must be integer, etc...

The default PILe class is as follow:

    @dataclass
    class PILe:    
        coords = (np.asarray([0]), np.asarray([0])) # one object at (0, 0), centre of the image
        height = 2              # see below
        width = 2
        shapes = 'circle'       # A circle
        sizes = 20              # of size 20
        alphas = 255            # fully visible
        colors = (255, 0, 0)    # red
        outlines = (0, 0, 0)    # with black outline
        widths = 3              # of width 3
        angles = 0              # not rotated
        ratios = 1              # height & width of the object are the same
        
        images =  Image.new("RGBA", (10, 10), (255, 255, 255, 0)) # A transparent 10*10 image, to be used by DrawImages


Note the height and width of 2: as the x and y coordinates will be scaled by PILe.width/2 + img.width/2. To enter 2 allows to simply enter the coordinates of the points wherever we want, keeping the centre of the image as (0, 0). Any number other than 2 will scale the coordinates to fit in a (width, height) rectangle in the centre of the image.  

When using the default PILe class, it should generate a single red disk with dark outline, in the centre of the image:

In [ ]:
# First, creating a regular PIL image
img1 = Image.new("RGBA", (50, 50), (255, 255, 255, 255))

default = PILe()
# Calling ImageDraws from PILes for multiple drawings
drawer = ImageDraws(img1) 
# And DrawShapes from PILes
drawer.DrawShapes(default) 

display(img1)

We can use it to draw multiple shapes on a single image at any coordinates. However, rememeber the (0,0) reference is at the centre of the image.
As we enter list of values, they will be cycled to reach the proper length

In [ ]:
mult = PILe()
mult.coords = [-20, 0, 0, 20], [0, 20, -20, 0] # We can add coordinates, (0,0) is the centre of the image
mult.sizes = [5, 10, 15, 10] # Different sizes of objects
mult.widths = [1, 2, 4, 0] # Different widths of contours
mult.colors = ['red', 'green', 'blue', 'grey'] # Different colors
mult.shapes = ['circle', 'rectangle']  # Different shapes.
mult.ratios = [1, 0.8, 1, 2] # ratios define the width/height ratio of the shape. For rectangles and ellipses.

img2 = Image.new("RGBA", (100, 100), (255, 255, 255, 2555))
drawer = ImageDraws(img2)
drawer.DrawShapes(mult)
display(img2)

By using the distributions module, we can easily generate multiple coordinates and simply pass them to the PILe dataclass:

In [ ]:
rosetta = PILe() # Let's draw a simple rosetta.
rosetta.coords = Circular(n=6).uniform(turns=1) # This will produce 6 points, uniformly spread on a circle.
rosetta.height, rosetta.width = 100, 100 # These are the dimensions for the centres of the circles
rosetta.shapes = 'circle' # We will draw circles
rosetta.sizes = 50 # Half the size of the image for regular rosetta
rosetta.colors = None # No colors will keep the shape empty
rosetta.outlines = 'black'
rosetta.widths = 4

# With centres in a 100*100 pixels area and radius of 50, the image will be 200 pixels wide and length
img3 = Image.new("RGBA", (210, 210), (255, 255, 255, 255))
drawer = ImageDraws(img3)
drawer.DrawShapes(rosetta)
display(img3)

We can use the colormaps made with ColorPiles as individual shape color:

In [ ]:
disk = PILe()
disk.height, disk.width = 600, 600
x, y = RandomCoords(n=1000).disk() # Just 1000 circles in a random disk distribution
disk.coords = x, y
disk.sizes = 20 # Single size
distances = Distance(x, y).inc_uniform() # This will generate an array of values in [0, 1] depending on distance
disk.colors = cmap3.map_to_index(distances) # Using the previous cmap3 indexed by distance.
disk.alphas = 128 # Adding transparency to each disk. Alphais between 0 and 255
disk.widths = 0 # No contour

img4 = Image.new("RGBA", (800, 800), (255, 255, 255, 255))
drawer = ImageDraws(img4)
drawer.DrawShapes(disk)
display(img4)

We can play with colors, contours, alphas, sizes, etc...

In [ ]:
sunflower = PILe()
sunflower.height, sunflower.width = 600, 600
x, y = Parametric(n=500).sunflower()
sunflower.coords = x, y

# As functions outputs weights between 0 and 1, we have to scale the sizes to pixels.
sunflower.sizes = Distance(x, y).wave(waves=2) * 10 + 8

distances = Distance(x, y).inc_uniform()

sunflower.colors = Chatoyant.ColorMap().from_matplotlib('RdBu', n=20).map_to_index(distances)
sunflower.alphas = [int(x) for x in Angle(x, y).wave(waves=5) * 255]
# Note that I did not specify any contours, but it is 3 pixels by default

img5 = Image.new("RGBA", (800, 800), (255, 255, 255, 255))
drawer = ImageDraws(img5)
drawer.DrawShapes(sunflower)
display(img5)

And play with shapes as well. Accepted shapes are triangle, square, rectangle, pentagon, hexagon, heptagon, octagon and any regular shape called 'n-gon'. A 3-gon and a triangle are equivalent.

In [ ]:
spiral2 = PILe()
spiral2.height, spiral2.width = 600, 600
x2, y2 = Spiral(n=200).quadratic(k=11)
spiral2.coords = x2, y2
spiral2.shapes = 'hexagon'

# Again, scaling the weights from [0, 1] to the wanted sizes
spiral2.widths = (- Distance(x2, y2).normal() * 5) + 6
spiral2.sizes = - Distance(x2, y2).normal() * 20 + 30

distances = Distance(x2, y2).inc_uniform()

spiral2.colors = Chatoyant.ColorMap().from_matplotlib('inferno', 20).map_to_index(distances)
spiral2.outlines = Chatoyant.ColorMap().from_matplotlib('viridis', 10).invert().map_to_index(distances)
spiral2.angles = Angle(x2, y2).wave() * 45
spiral2.alphas = 255

img6 = Image.new("RGBA", (800, 800), (128, 128, 128, 0))
drawer = ImageDraws(img6)
drawer.DrawShapes(spiral2)

display(img6)

All attributes can be indexed by any array, used as an index.
Either colormaps:

In [ ]:
tiles = PILe()

tiles.height, tiles.width = 600, 600
x, y = Uniform(n=200).square()
tiles.coords = x, y
tiles.shapes = 'square'
# widths, sizes and angles will change through distance from centre of distribution
# We have to make sure the widths and sizes are positive integers in the end.
tiles.widths = (- Distance(x, y).normal() * 5) + 6
tiles.sizes = - Distance(x, y).normal(sd=0.6) * 25 + 30
tiles.angles = Distance(x, y).wave(waves=1) * 45 

# colors will be indexed by x or y axis.
tiles.colors = Chatoyant.ColorMap().from_matplotlib('RdBu', 12).map_to_index(x)
tiles.outlines = Chatoyant.ColorMap().from_matplotlib('RdYlBu', 12).invert().map_to_index(y)

tiles.alphas = 255

img7 = Image.new("RGBA", (800, 800), (128, 128, 128, 0))
drawer = ImageDraws(img7)
drawer.DrawShapes(tiles)

display(img7)


We can index sizes, widths, etc... as well:

In [ ]:
tiles2 = PILe()

tiles2.height, tiles2.width = 400, 400
x, y = Uniform(n=300).hexagon()

# x and y are numpy arrays, so we can manipulate them easily
mask = (x**2 + y**2) <= 1
x = x[mask]
y = y[mask]

# Let's add some jitter in the coordinates with the 'Modify' class.
tiles2.coords = Modify(x, y).jitter(x = 0, y = 0.1)
tiles2.shapes = 'circle'
# widths and sizes and angles will change through axis
# We need to normalize the values between 0 and 1 first. 
norm_x, norm_y = Modify(x, y).normalize() 
tiles2.widths = norm_x * 5 + 1
tiles2.sizes = norm_y * 15 + 5

# colors will be indexed by x or y axis.
tiles2.colors = Chatoyant.ColorMap().from_matplotlib('RdBu', 12).map_to_index(x)
tiles2.outlines = Chatoyant.ColorMap().from_matplotlib('RdYlBu', 12).invert().map_to_index(x)

tiles2.alphas = 255
tiles2.angles = 0

img8 = Image.new("RGBA", (450, 450), (128, 128, 128, 0))
drawer = ImageDraws(img8)
drawer.DrawShapes(tiles2)

display(img8)

# DrawLines
The module comes with a 'DrawLines' method, which will draw a line between (x, y) pairs of coordinates.
It uses the same dataclass and can be used with any previous PILe object. Example by drawing the lines between the centres of the rosetta:

In [ ]:
img4 = Image.new("RGBA", (210, 210), (255, 255, 255, 255))
drawer = ImageDraws(img4)
drawer.DrawShapes(rosetta) # First we draw the rosetta

# Drawlines will simply draw a line between each (x, y) pair of coordinates.
# closed = True will close the shape, drawing a line between the first and last point
drawer.DrawLines(rosetta, closed=True) # that's all.

display(img4)

The DrawLines method comes with 2 keyword arguments: continuous=True and closed=False by default.
They allow to control the behaviour of the method.
continous = True will draw a line between each point one by one, when continuous = False will draw a line every two points.
closed = True will draw a line between the last and first point, when closed = False will stop at the last point.

As an example:

In [ ]:
# Let's create a Spiral distribution.
spiral = PILe() 
spiral.coords = Spiral(n=50).golden() 
spiral.height, spiral.width = 300, 300
spiral.outlines = 'black'
spiral.widths = 8


Let's create 3 images from this distribution, with different parameters.

In [ ]:
img3 = Image.new("RGBA", (310, 310), (255, 255, 255, 255))
drawer = ImageDraws(img3)
drawer.DrawLines(spiral, closed=False) # First image will be an open shape

img4 = Image.new("RGBA", (310, 310), (255, 255, 255, 255))
drawer = ImageDraws(img4)
drawer.DrawLines(spiral, closed=True) # Second Image will show a closed shape

img5 = Image.new("RGBA", (310, 310), (255, 255, 255, 255))
drawer = ImageDraws(img5)
drawer.DrawLines(spiral, continuous=False) # Third Image will have only 1 line every 2.


To better present multiple PIL pictures, we can use GroupImages:

In [ ]:
from piles import GroupImages

final = GroupImages([img3, img4, img5]).multidraw(ncols=3, nrows=1, border = 10, titles=['Open', 'Closed', 'Alternate'])
display(final)

We can make complex drawings with the DrawLines method as well:

In [ ]:
spiral3 = PILe()
spiral3.height, spiral3.width = 400, 400
x3, y3 = Spiral(n=500).quadratic(k=17)
spiral3.coords = x3, y3

spiral3.widths = (- Distance(x3, y3).normal() * 5) + 10

distances = Distance(x3, y3).inc_uniform()

# Playing with outlines colors and not fill colors.
spiral3.outlines = Chatoyant.ColorMap().from_matplotlib('seismic', 50).map_to_index(distances) 

spiral3.alphas = Angle(x3, y3).wave(waves=5) * 128

img7 = Image.new("RGBA", (450, 450), (128, 128, 128, 0))
drawer = ImageDraws(img7)
drawer.DrawLines(spiral3)

display(img7)

### Note
PIL does not manage anti-aliasing. To create it, best method is to generate big images and then downsize them.
As an example, if we take the last image generated:


In [ ]:
spiral3 = PILe()
spiral3.height, spiral3.width = 1200, 1200 # Let's make it BIG! Multiplying dimensions by 3.
x3, y3 = Spiral(n=500).quadratic(k=17)
spiral3.coords = x3, y3

spiral3.widths = (- Distance(x3, y3).normal() * 15) + 30 # Simply multiply everything by 3
distances = Distance(x3, y3).inc_uniform()

# Playing with outlines colors and not fill colors.
spiral3.outlines = Chatoyant.ColorMap().from_matplotlib('seismic', 50).map_to_index(distances) 
spiral3.alphas = Angle(x3, y3).wave(waves=5) * 128

img8 = Image.new("RGBA", (1350, 1350), (128, 128, 128, 0)) # canvas size was multiplied by 3 as well.
drawer = ImageDraws(img8)

drawer.DrawLines(spiral3) 

img9 = img8.resize ((450, 450)) # And then downsize it to original size

comparison = GroupImages([img7, img9]).multidraw(nrows=1, ncols=2, titles=['Small size', 'Big size, downscaled'])

display(comparison) # Note the anti-aliased image compared to previous one.

Aliasing disappeared due to the downsizing. Note that it takes longer to draw big images, so it might be wise to do trials on small ones only.

# DrawImages

The module can work with images as well, repeating them at the desired coordinates.
The only difference is the parameter 'sizes', as it will be a multiplier of the original size of the image. It should be set to 1 for keeping the original size of the image.

In [ ]:
# First, creating a regular PIL image
tile = Image.new("RGBA", (100, 100), (255, 255, 255, 0))

tile_pict = PILe()
# We will draw shapes wherever we want, hence the values of 2.
tile_pict.width, tile_pict.height = 2, 2
tile_pict.coords = ([0, 0, 0, 0, 0], [0, -10, -16, -12, -10])
tile_pict.shapes = ['ellipse']
tile_pict.colors = ['light_sea_green', 'orange', 'blue', 'black', 'light_blue']
tile_pict.sizes = [40, 28, 20, 12, 6]
tile_pict.ratios = [0.8, 0.9, 1, 1.2, 1.5]

tile_pict.widths = 0
tile_pict.angles = 0 
# We draw the tile in a small image
drawer = ImageDraws(tile) 
drawer.DrawShapes(tile_pict) 

# Here is the tile
display(tile)

In [ ]:
# Now we will distribute this image with a new PILe object:
img2 = Image.new("RGBA", (800, 800), (255, 255, 255, 0))

pattern = PILe()
pattern.height, pattern.width = 600, 600
# Distribution on a hexagonal lattice
x, y = Parametric(n=100).sunflower()
pattern.coords = x, y

# We simply put the previous image in the PILe Dataclass

pattern.images = tile
pattern.alphas = 255

# Images will be rotated along the x axis
# Angles follow radian notation, so starting at the right and going counter-clockwise. 
# And the image has to be rotated 90 degrees first.
pattern.angles = 90 + Angle(x, y).inc_uniform() * 360

# Size will vary according to the distance from the distribution centre, from half to full size.
pattern.sizes = Distance(x, y).inc_uniform() * 0.5 + 0.5

# And we draw the images with DrawImages
drawer = ImageDraws(img2)
drawer.DrawImages(pattern)

display(img2)

## Other
#### Perlin Noise and weights according to neighbours.
We can either draw Perlin noise, or use it as weights according to number of neighbours, etc...

In [ ]:
from distributions import Noise
from functions import Weights

noise = PILe()

x, y = RandomCoords(5000).disk() # Random points in a disk shape.
noise.coords = x, y

a, b = Noise(steps=300, amount=2).perlin(grid=10) # Perlin noise

noise.width, noise.height = 500, 500
noise.widths = 0

# Calculate how many (a, b) neighbours have points in (x, y).
# If we remove "a, b", it will calculate the number of neighbours of x and y.
n = Weights(x, y).n_neighbors(a, b, dist=0.2)

# Sizes and colors of the (x, y) points will be dependent on their number of neighbors on (a, b).
# So sizes and colors will be dependent of the perlin noise.
noise.sizes = n * 5 + 2
noise.colors = Chatoyant.ColorMap().from_matplotlib('Spectral', n=100).map_to_index(n)

noise_img = Image.new("RGBA", (500, 500), (255, 255, 255, 0))
drawer = ImageDraws(noise_img)
drawer.DrawShapes(noise)

display(noise_img)

#### Image to PILe
This function will extract the shapes of a PILe from an image, to generate distributions with images.

In [ ]:
from piles import ImageOps

pix = Image.open(r'Portal_Companion_Cube.jpg').convert('RGBA') # Our best talking friend

# Just doing a basic PILe with sunflower sistribution
sunflower = PILe()
sunflower.width, sunflower.height = pix.size
x, y = Parametric(n=500).sunflower()
sunflower.coords = x, y
sunflower.sizes = Distance(x, y).wave(waves=2) * 10 + 5
sunflower.widths = 0
sunflower.alphas = 255
sunflower.colors = 'black'
sunflower.shapes = 'triangle'
sunflower.ratios = 1

# And draw it to show what it looks like
sun_img = Image.new("RGBA", pix.size, (255, 255, 255, 255))
drawer = ImageDraws(sun_img)
drawer.DrawShapes(sunflower)

# Let's combine the PILe with the image to extract all the tiles.
tiles = ImageOps(pix).to_pile_images(sunflower)
# And show the result
tile_img = Image.new("RGBA", pix.size, (255, 255, 255, 255))
drawer = ImageDraws(tile_img)
# Use DrawImages to draw the tiles and not the shapes.
drawer.DrawImages(tiles)

# We can now modify this PILe object, for example introducing angle noise
tiles.angles = np.random.uniform(-60, 60, len(tiles.coords[0]))
# And draw the modified result.
tile_img2 = Image.new("RGBA", pix.size, (255, 255, 255, 255))
drawer = ImageDraws(tile_img2)
drawer.DrawImages(tiles)

# Let's show everything.
GroupImages([pix, sun_img, tile_img, tile_img2]).multidraw(ncols=2, nrows=2, 
                                                           titles=['Image', 'PILe', 
                                                                   'Pile Images', 'Modified PIle'])

#### Combination with TILEs
The module TILEs (https://github.com/Sylvain-Deposit/TILEs) will decompose the image in quadtrees, voronoitrees, etc...
It returns a dict of all extracted components: polygons, colors, images, and other that can be then used by PILes to modify them.

In [ ]:
from TILEs import quadtree
pix = Image.open(r'Portal_Companion_Cube.jpg')

# quad_img is the new quadtree image.
# quad_pile is the PILe object
quad_dict = quadtree(pix, std_thr=5)

quad_pile = PILe()

x = np.asarray(quad_dict['x']) - (pix.width / 2)
y = np.asarray(quad_dict['y']) - (pix.height / 2)
s = np.asarray(quad_dict['height']) / 2
r = np.asarray(quad_dict['width']) / np.asarray(quad_dict['height'])
c = quad_dict['colors']

quad_pile.coords = x, -y  # -y as images y coords are flipped
quad_pile.sizes = s
quad_pile.ratios = r
quad_pile.colors = c
quad_pile.shapes = 'rectangle'
quad_pile.angles = 0
quad_pile.widths = 0

# Let's draw the PILe object in a new image. It will be the quadtree image
img3 = Image.new("RGBA", (pix.size), (255, 255, 255, 255))
drawer = ImageDraws(img3)
drawer.DrawShapes(quad_pile)

# We can modify the PILe object: replace the shapes by triangles, add some rotation noise, etc...
quad_pile.shapes = 'triangle'
quad_pile.angles = np.random.uniform(-90, 90, len(quad_pile.coords[0]))
quad_pile.sizes = quad_pile.sizes * 3

# We have to sort the shapes in descending order, to draw the small shapes on top of the others.
quad_pile.sort(sort=['coords', 'sizes', 'colors', 'ratios'], by='sizes', descending=True)

# And draw the new PILe object
img4 = Image.new("RGBA", (pix.size), (255, 255, 255, 255))
drawer = ImageDraws(img4)
drawer.DrawShapes(quad_pile)

# Let's show the outlines of the polygons
quad_pile.widths = 1
img5 = Image.new("RGBA", (pix.size), (255, 255, 255, 255))
drawer = ImageDraws(img5)
drawer.DrawShapes(quad_pile)

# And display everything
final = GroupImages([pix, img3, img4, img5]).multidraw(nrows=2, ncols=2, 
                                               titles=['Original', 'PILe from Quadtree', 
                                                       'Modified PILe', 'Modified PILe with outlines'])
display(final)

